# 05 Minimal Relevance Closure

This notebook defines a method to calculate if a statement (a twiddle statement) is entailed by the minimal relevance closure of a knowledge base.

In [1]:
import os
import sys
sys.path.append(os.path.join('..', 'common'))

from datatypes import KnowledgeBase, Literal, Atom, Top, Bot, Normally, Formula
from util import materialized, print_knowledge_base, entails
from justifications import justifications, separate
from ranked_models import statement_ranking

In [2]:
bot = Bot()            # Falsum
top = Top()            # Verum
m = Literal(Atom('m')) # mamalian red blood cells
v = Literal(Atom('v')) # vertebrate red blood cells
a = Literal(Atom('a')) # avian red blood cells
c = Literal(Atom('c')) # cell membrane
n = Literal(Atom('n')) # nucleus
s = Literal(Atom('s')) # mammalian sickle cells
b = Literal(Atom('b')) # bioconcave shape

We deploy the same justification algorithm, however it is omitted here.

Instead of deleting all statements in the justifications, we only delete one (the minimum accoring to the statement rank) per each justification.

In [3]:
def minimal_relevance_closure(knowledge_base: KnowledgeBase, statement: Normally) -> bool:
    rank = statement_ranking(knowledge_base)
    inverse_rank = {s:k for k,v in rank.items() for s in v}
    i = 0
    T, D = separate(knowledge_base)
    J = justifications(T, D, statement.left)
    Rel = {min(js, key=inverse_rank.get) for js in J}
    Irr = D - Rel
    R_ = materialized(Rel)
    while R_ and entails(materialized(T) | materialized(Irr) | R_, -statement.left):
        R_ = R_ - materialized(rank[i] & Rel)
        i += 1
    return entails(materialized(T) | materialized(Irr) | R_, statement.materialize())

In [4]:
K={-(m >> v) / bot, -(a >> v) / bot, v / c, v / n, m / -n, -(s >> m) / bot, m / b, s / -b}
T, D = separate(K)
rank = statement_ranking(K)
inverse_rank = {s:k for k,v in rank.items() for s in v}
print_knowledge_base(K)

{ m → v, m |~ ¬n, s → m, v |~ n, m |~ b, s |~ ¬b, a → v, v |~ c }


In [5]:
J = justifications(T, D, Formula(s))
Rel = {min(js, key=inverse_rank.get) for js in J}
Rel

{m |~ b, v |~ n}

In [6]:
minimal_relevance_closure(K, s / -n)

True